In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
mov_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')
rat_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv')
gen_tag_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/genome_tags.csv')
gen_sco_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/genome_scores.csv')
tag_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/tag.csv')
link_df = pd.read_csv('/kaggle/input/movielens-20m-dataset/link.csv')


In [ ]:
mov_rat = pd.merge(mov_df,rat_df)
mov_rat

In [ ]:
gen_sco_tag = pd.merge(gen_tag_df,gen_sco_df)
gen_sco_tag.head()

In [ ]:
tag_df.head()
tag_df = tag_df.drop(['timestamp'],axis=1)

In [ ]:
gen_sco_tag = pd.merge(gen_sco_tag,tag_df)

In [ ]:
gen_sco_tag

this looks better with all tags sorted together, I will sort it into one big datframe later

In [ ]:
link_df.head()

Guess i wont be needing this

So now we have the neccesary data and thus we will combine it and do some preprocessing

In [ ]:
df = pd.merge(mov_rat,gen_sco_tag)
df.head()

In [ ]:
df = df.drop(['timestamp','tagId','tag','relevance'],axis=1)
df

In [ ]:
df['rating']=df['rating'].astype(int)
df

In [ ]:
number_rating = mov_rat.groupby('title')['rating'].count().reset_index()

In [ ]:
number_rating.rename(columns={'rating':'number of rating'},inplace=True)
number_rating.head()

In [ ]:
df = pd.merge(df,number_rating)
df

In [ ]:
df.shape

In [ ]:
df=df[df['number of rating']>=50]

In [ ]:
df.drop_duplicates(['title','userId'],inplace=True)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.drop(columns=['number of rating'],inplace=True)
df.head()

In [ ]:
mov_piv=df.pivot_table(columns='userId',index='title',values='rating')

In [ ]:
mov_piv.shape

In [ ]:
mov_piv.fillna(0,inplace=True)

In [ ]:
mov_piv

In [ ]:
from scipy.sparse import csr_matrix
mov_spar=csr_matrix(mov_piv)

In [ ]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=10,algorithm='brute',metric='cosine')

In [ ]:
knn.fit(mov_spar)

In [ ]:
df.drop(columns=['genres','userId','rating'],inplace=True)


In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.to_csv('RecoSys.csv',index=False)

In [ ]:
df_f =df.copy()
fin=[]
for i in df_f['title']:
    fin.append(i.split(' (')[0])
df_f['title']=fin

In [ ]:
def reco(mov_name):
    movie_id=df_f[df_f['title']=='Toy Story'].drop_duplicates('title')['movieId'].values[0]
    distances,suggestions=knn.kneighbors(mov_piv.iloc[movie_id,:].values.reshape(1,-1))
    
    
    
    for i in range(len(suggestions)):
        return (mov_piv.index[suggestions[i]])

In [ ]:
res=reco('xXx(2002)')

In [ ]:
for i in res:
    print(i)